In [1]:
import gymnasium as gym
import numpy as np
import math
import random
from collections import defaultdict

In [2]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = {}
        self.visits = 0
        self.value = 0.0

    def is_fully_expanded(self, action_space):
        return len(self.children) == action_space.n

    def best_child(self, exploration_constant=math.sqrt(2)):
        choices_weights = [
            (child.value / (child.visits + 1e-10)) + exploration_constant * math.sqrt(
                math.log(self.visits + 1) / (child.visits + 1e-10)
            )
            for child in self.children.values()
        ]
        return list(self.children.values())[np.argmax(choices_weights)]

    def select_best_action(self):
        best_action = max(self.children.items(), key=lambda item: item[1].visits)[0]
        return best_action




In [3]:
class MCTS:
    def __init__(self, env, num_simulations=1000, exploration_constant=math.sqrt(2)):
        self.env = env
        self.num_simulations = num_simulations
        self.exploration_constant = exploration_constant

    def search(self, initial_state):
        root = Node(initial_state)

        for _ in range(self.num_simulations):
            node = root
            state = initial_state

            # Selection
            while not node.is_fully_expanded(self.env.action_space) and node.children:
                node = node.best_child(self.exploration_constant)
                action = list(node.parent.children.keys())[list(node.parent.children.values()).index(node)]
                state, _, done, _, _ = self.env.step(action)
                if done:
                    break

            # Expansion
            if not node.is_fully_expanded(self.env.action_space) and not done:
                action = random.choice([a for a in range(self.env.action_space.n) if a not in node.children])
                next_state, reward, done, _, _ = self.env.step(action)
                node.children[action] = Node(next_state, parent=node)
                node = node.children[action]

            # Simulation
            reward = self.rollout(state)

            # Backpropagation
            while node is not None:
                node.visits += 1
                node.value += reward
                node = node.parent

        return root.select_best_action()

    def rollout(self, state):
        done = False
        total_reward = 0
        while not done:
            action = self.env.action_space.sample()
            state, reward, done, _, _ = self.env.step(action)
            total_reward += reward
        return total_reward



In [4]:
env = gym.make('FrozenLake-v1', is_slippery=True)
mcts = MCTS(env)

episodes = 100
rewards = []

for episode in range(episodes):
    state = env.reset()[0]
    done = False
    total_reward = 0
    
    while not done:
        action = mcts.search(state)
        state, reward, done, _, _ = env.step(action)
        total_reward += reward
    
    rewards.append(total_reward)

plt.plot()
print(f'Average Reward over {episodes}: {np.mean(rewards)}')


UnboundLocalError: local variable 'done' referenced before assignment